In [1]:
from torchvision.models import resnet18
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
from tqdm import tqdm
import torchvision.transforms as transforms
from torchvision import datasets, transforms
from tqdm import tqdm 
import time
from torch.utils.data import random_split, Dataset, DataLoader
from torchvision.models.resnet import ResNet18_Weights
import torchviz
import pickle
import random

seed = 42
torch.manual_seed(seed)

In [13]:
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

### Load Resent Model 

In [3]:
def get_cifar10_model():
    model = torchvision.models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)  # Use 'weights' for pretrained models
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 10)
    return model

In [4]:
model = get_cifar10_model()
print(type(model))

<class 'torchvision.models.resnet.ResNet'>


In [8]:
# Freeze pre-trained layers
for param in model.parameters():
    param.requires_grad = False

# Unfreeze some layers for fine-tuning
for param in model.layer4.parameters():
    param.requires_grad = True

In [8]:
# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

In [3]:
# Define data transforms
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

In [7]:
# Load CIFAR10 datasets
train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform_test)

# Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


## LeNet Model Definition

In [4]:
class LeNet(nn.Module):    
    def __init__(self):
        super(LeNet, self).__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(400,120),  #in_features = 16 x5x5 
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,10),
            nn.Softmax(dim=1)

        )
        
    def forward(self,x): 
        a1=self.feature_extractor(x)
        # print(a1.shape)
        a1 = torch.flatten(a1,1)
        a2=self.classifier(a1)
        return a2

# Baseline Model Training

### Basic Train Loop

In [9]:
# Define the Model
# model = get_cifar10_model()
model = LeNet()
model = model.to(device)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

# Train the model
epochs = 20
model.train()
start_time = time.time()
for epoch in tqdm(range(epochs)):
    # Train loop
    for images, labels in tqdm(train_dataloader):

        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        
        # Backward pass and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Evaluate on test set
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(test_dataloader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            predictions = torch.argmax(outputs, dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    print(f"Epoch: [{epoch+1}/{epochs}], Accuracy: {accuracy:.4f}")

print("--- %s seconds ---" % (time.time() - start_time))

  5%|▌         | 1/20 [00:38<12:08, 38.35s/it]

Epoch: [1/20], Accuracy: 0.3563


 10%|█         | 2/20 [01:12<10:49, 36.09s/it]

Epoch: [2/20], Accuracy: 0.4079


 15%|█▌        | 3/20 [01:46<09:56, 35.10s/it]

...

Epoch: [18/20], Accuracy: 0.5386


 95%|█████████▌| 19/20 [10:08<00:30, 30.38s/it]

Epoch: [19/20], Accuracy: 0.5402


100%|██████████| 20/20 [10:39<00:00, 31.95s/it]

Epoch: [20/20], Accuracy: 0.5476
--- 639.0781149864197 seconds ---


# Reweight Model Training

In [6]:
# Load CIFAR10 datasets
train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform_test)

split_ratio = 0.95 # 80% for test, 20% for validation
n_samples = len(test_dataset)
n_test = int(n_samples * split_ratio)
n_val = n_samples - n_test
test_dataset, val_dataset = random_split(test_dataset, [n_test, n_val])


# Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


### Random Sampler for Sampling Validation Data

In [23]:
class RandomSubsetSampler(torch.utils.data.Sampler):
    def __init__(self, dataset, subset_size):
        self.dataset = dataset
        self.subset_size = subset_size

    def __iter__(self):
        indices = random.sample(range(len(self.dataset)), self.subset_size)
        return iter(indices)

    def __len__(self):
        return self.subset_size

subset_sampler = RandomSubsetSampler(val_dataset, 64)
subset_dataloader = DataLoader(val_dataset, sampler=subset_sampler)

NameError: name 'val_dataset' is not defined

### Meta Baseline Trainloop

In [39]:
# Define the Model
# model = get_cifar10_model()
model = LeNet()
model = model.to(device)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
loss_fn_meta = nn.CrossEntropyLoss(reduction='none')

# Train the model
epochs = 20
lr = 0.001
model.train()
start_time = time.time()
for epoch in tqdm(range(epochs)):
    # Train loop
    for images, labels in tqdm(train_dataloader):
        
        images = images.to(device)
        labels = labels.to(device)

        # meta_net = get_cifar10_model()
        meta_net = LeNet()
        meta_net.load_state_dict(model.state_dict())

        meta_net = meta_net.to(device)

        optimizer_meta = torch.optim.Adam(meta_net.parameters())

        meta_net.train()
        
        y_f_hat = meta_net(images)
        cost = loss_fn_meta(y_f_hat, labels)
        eps = torch.zeros(cost.size(), requires_grad=True).to(device)
        l_f_meta = torch.sum(cost*eps)

        # meta_net.zero_grad()
        optimizer_meta.zero_grad()
        eps.retain_grad()
        l_f_meta.backward()
        optimizer_meta.step()

        meta_net.eval()

        # grads = torch.autograd.grad(l_f_meta, (meta_net.parameters()), create_graph=True)
        # meta_net.update_params(lr, source_params=grads)
        
        val_images, val_labels = next(iter(subset_dataloader))
        # val_images, val_labels = next(iter(val_dataloader))
        val_images = val_images.to(device)
        val_labels = val_labels.to(device)

        y_g_hat = meta_net(val_images)
        l_g_meta = loss_fn(y_g_hat, val_labels)

        # grad_eps = torch.autograd.grad(l_g_meta, eps, only_inputs=True)[0]
        # grad_eps = torch.autograd.grad(l_g_meta, eps, only_inputs=True, allow_unused=True)[0]
        # print(grad_eps)

        with torch.no_grad():
            optimizer_meta.zero_grad()
            l_g_meta.backward()
            grad_eps = eps.grad
        
        # print(grad_eps)
        w_tilde = torch.clamp(grad_eps,min=0)
        # w_tilde = torch.clamp(-grad_eps,min=0)
        norm_c = torch.sum(w_tilde)

        if norm_c != 0:
            w = w_tilde / norm_c
        else:
            w = w_tilde
        
        # print(w)
        # break
        # Forward Pass
        outputs = model(images)
        loss = loss_fn_meta(outputs, labels)
        loss = torch.sum(loss*w)
        
        # Backward pass and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Evaluate on test set
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(test_dataloader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            predictions = torch.argmax(outputs, dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    print(f"Epoch: [{epoch+1}/{epochs}], Accuracy: {accuracy:.4f}")

print("--- %s seconds ---" % (time.time() - start_time))

  5%|▌         | 1/20 [00:50<16:06, 50.86s/it]

Epoch: [1/20], Accuracy: 0.3925


 10%|█         | 2/20 [01:40<15:06, 50.37s/it]

Epoch: [2/20], Accuracy: 0.4705


 15%|█▌        | 3/20 [02:31<14:17, 50.45s/it]

...

Epoch: [18/20], Accuracy: 0.5984


 95%|█████████▌| 19/20 [17:02<00:51, 51.32s/it]

Epoch: [19/20], Accuracy: 0.5919


100%|██████████| 20/20 [17:52<00:00, 53.62s/it]

Epoch: [20/20], Accuracy: 0.6049
--- 1072.4640719890594 seconds ---


# Milo Setup

### Load Data

In [17]:
num_classes = 10
class_data = []
for i in range(num_classes):
    with open(f"milo-base/class-data/class_{i}.pkl", "rb") as f:
        S = pickle.load(f)
        class_data.append(S)

In [16]:
num_sets = len(class_data[0])
data = []
for i in range(num_sets):
    S = []
    for j in range(num_classes):
        S.extend(class_data[j][i])
    data.append(S)

### Define Dataloader

In [15]:
class SubDataset(Dataset):
    def __init__(self, indices, dataset):
        self.indices = indices
        self.dataset = dataset

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        index = self.indices[idx]
        data_point = self.dataset[index]
        return data_point

## Milo Training Loop

In [21]:
# Define Model
model = LeNet()
model = model.to(device)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

# Train the model
epochs = 20
model.train()
start_time = time.time()
for epoch in tqdm(range(epochs)):
    # Train loop

    sub_dataset = SubDataset(indices=data[epoch], dataset=train_dataset)
    subset_train_dataloader = DataLoader(sub_dataset, batch_size=64, shuffle=True)
    
    for images, labels in tqdm(subset_train_dataloader):

        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        
        # Backward pass and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Evaluate on test set
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(test_dataloader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            predictions = torch.argmax(outputs, dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    print(f"Epoch: [{epoch+1}/{epochs}], Accuracy: {accuracy:.4f}")

print("--- %s seconds ---" % (time.time() - start_time))

  5%|▌         | 1/20 [00:13<04:25, 13.98s/it]

Epoch: [1/20], Accuracy: 0.2393


 10%|█         | 2/20 [00:27<04:06, 13.71s/it]

Epoch: [2/20], Accuracy: 0.2537


 15%|█▌        | 3/20 [00:41<03:51, 13.63s/it]

...

Epoch: [18/20], Accuracy: 0.4113


 95%|█████████▌| 19/20 [04:20<00:13, 13.68s/it]

Epoch: [19/20], Accuracy: 0.4032


100%|██████████| 20/20 [04:34<00:00, 13.71s/it]

Epoch: [20/20], Accuracy: 0.4157
--- 274.18256092071533 seconds ---


In [25]:
# Load CIFAR10 datasets
train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform_test)

split_ratio = 0.95 # 80% for test, 20% for validation
n_samples = len(test_dataset)
n_test = int(n_samples * split_ratio)
n_val = n_samples - n_test
test_dataset, val_dataset = random_split(test_dataset, [n_test, n_val])


# Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [26]:
subset_sampler = RandomSubsetSampler(val_dataset, 64)
subset_dataloader = DataLoader(val_dataset, sampler=subset_sampler)

### Meta-Milo Training loop

In [27]:
# Define Model
model = LeNet()
model = model.to(device)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
loss_fn_meta = nn.CrossEntropyLoss(reduction='none')

# Train the model
epochs = 20
lr = 0.001
model.train()
start_time = time.time()
for epoch in tqdm(range(epochs)):
    # Train loop
    sub_dataset = SubDataset(indices=data[epoch], dataset=train_dataset)
    # train_dataloader = DataLoader(sub_dataset, batch_size=64, shuffle=True, num_workers=2)
    train_dataloader = DataLoader(sub_dataset, batch_size=64, shuffle=True)
    
    for images, labels in tqdm(train_dataloader):
        
        images = images.to(device)
        labels = labels.to(device)

        # meta_net = get_cifar10_model()
        meta_net = LeNet()
        meta_net.load_state_dict(model.state_dict())

        meta_net = meta_net.to(device)

        optimizer_meta = torch.optim.Adam(meta_net.parameters())

        meta_net.train()
        
        y_f_hat = meta_net(images)
        cost = loss_fn_meta(y_f_hat, labels)
        eps = torch.zeros(cost.size(), requires_grad=True).to(device)
        l_f_meta = torch.sum(cost*eps)

        # meta_net.zero_grad()
        optimizer_meta.zero_grad()
        eps.retain_grad()
        l_f_meta.backward()
        optimizer_meta.step()

        meta_net.eval()

        # grads = torch.autograd.grad(l_f_meta, (meta_net.parameters()), create_graph=True)
        # meta_net.update_params(lr, source_params=grads)
        
        val_images, val_labels = next(iter(subset_dataloader))
        # val_images, val_labels = next(iter(val_dataloader))
        val_images = val_images.to(device)
        val_labels = val_labels.to(device)

        y_g_hat = meta_net(val_images)
        l_g_meta = loss_fn(y_g_hat, val_labels)

        # grad_eps = torch.autograd.grad(l_g_meta, eps, only_inputs=True)[0]
        # grad_eps = torch.autograd.grad(l_g_meta, eps, only_inputs=True, allow_unused=True)[0]
        # print(grad_eps)

        with torch.no_grad():
            optimizer_meta.zero_grad()
            l_g_meta.backward()
            grad_eps = eps.grad
        
        # print(grad_eps)
        w_tilde = torch.clamp(grad_eps,min=0)
        # w_tilde = torch.clamp(-grad_eps,min=0)
        norm_c = torch.sum(w_tilde)

        if norm_c != 0:
            w = w_tilde / norm_c
        else:
            w = w_tilde
        
        # print(w)
        # break
        # Forward Pass
        outputs = model(images)
        loss = loss_fn_meta(outputs, labels)
        loss = torch.sum(loss*w)
        
        # Backward pass and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Evaluate on test set
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(test_dataloader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            predictions = torch.argmax(outputs, dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    print(f"Epoch: [{epoch+1}/{epochs}], Accuracy: {accuracy:.4f}")

print("--- %s seconds ---" % (time.time() - start_time))

  5%|▌         | 1/20 [00:04<01:32,  4.89s/it]

Epoch: [1/20], Accuracy: 0.2315


 10%|█         | 2/20 [00:07<01:08,  3.81s/it]

Epoch: [2/20], Accuracy: 0.2860


 15%|█▌        | 3/20 [00:10<00:58,  3.45s/it]

...

Epoch: [18/20], Accuracy: 0.4200


 95%|█████████▌| 19/20 [00:59<00:03,  3.03s/it]

Epoch: [19/20], Accuracy: 0.4281


100%|██████████| 20/20 [01:02<00:00,  3.13s/it]

Epoch: [20/20], Accuracy: 0.4261
--- 62.580097913742065 seconds ---
